# Quick Start 

Welcome to the **ASL(Agent Stucture Language)** — A DSL(Domain Specific Language) about how to building an agent — documentation! This page will give you an introduction to 80% of the ASL usage that you will use on a daily basis.

> You will learn:
>   1. How to build an agent by ASL.
>   2. How to reuse the exit agents in a componentized parttern.
>   3. How to build an agent with a nested structure.
>   4. How to control data transmission in the agent
>   5. How to achieve dynamic topology during the agent's runtime.


## Introduction

ASL is a declarative language for agent construction. After all basic functions are modularly implemented, we pursue a what-you-see-is-what-you-get approach for the internal processes of the agent. We can clearly see the orchestration process and hierarchical structure at a glance.

### Build an Agent by ASL

Take the simplest text generation process as an example. When a user inputs a query, we break it down, and then generate text for every sub-query.

Let's first prepare necessary environment variables.

In [ ]:
# Get the environment variables.
import os

_api_key = os.environ.get("OPENAI_API_KEY")
_api_base = os.environ.get("OPENAI_API_BASE")
_model_name = os.environ.get("OPENAI_MODEL_NAME")

# Import the necessary packages.
from typing import List
from bridgic.core.model.types import Message, Role
from bridgic.core.agentic.asl import ASLAutoma, graph
from bridgic.llms.openai import OpenAILlm, OpenAIConfiguration

llm = OpenAILlm(  # the llm instance
    api_base=_api_base,
    api_key=_api_key,
    timeout=5,
    configuration=OpenAIConfiguration(model=_model_name),
)

Secondly, implement the functional functions needed in this process.

In [ ]:
# Break down the query into a list of sub-queries.
async def break_down_query(user_input: str) -> List[str]:
    llm_response = await llm.achat(
        messages=[
            Message.from_text(text=f"Break down the query into multiple sub-queries", role=Role.SYSTEM),
            Message.from_text(text=user_input, role=Role.USER,),
        ]
    )
    return llm_response.text.split("\n")

# Define the function to conduct a web search.
async def query_answer(queries: List[str]) -> str:
    answers = []
    for query in queries:
        response = llm.achat(
            messages=[
                Message.from_text(text=f"Answer the given query", role=Role.SYSTEM),
                Message.from_text(text=query, role=Role.USER,),
            ]
        )
        answers.append(response.text)
    
    res = {
        query: answer
        for query, answer in zip(queries, answers)
    }
    return res

Now, Let's complete this process using ASL.

In [ ]:
class TextGenerationAgent(ASLAutoma):
    pass
